In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Softmax, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import losses
from tensorflow.keras import Sequential
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score 
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.layers import Flatten,Dense

image_size=64
batch_size=256

In [4]:
image_size=32 
batch_size=256

In [5]:
image_data = []
image_labels = []

In [6]:
data_dir='/kaggle/input/waste-classification-data/DATASET'

In [7]:

path = data_dir + '/TRAIN/' + 'O'
images = os.listdir(path)
for img in images:
    image = cv2.imread(path + '/' + img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((image_size, image_size))
    image_data.append(np.array(resize_image))
    image_labels.append(0)
    
path = data_dir + '/TRAIN/' + 'R'
images = os.listdir(path)
for img in images:
    image = cv2.imread(path + '/' + img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((image_size, image_size))
    image_data.append(np.array(resize_image))
    image_labels.append(1)
# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)
print(image_data.shape, image_labels.shape)

(22564, 32, 32, 3) (22564,)


In [8]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

In [10]:
X_train,y_train=(image_data, image_labels)

In [11]:
X_train = X_train/255

In [12]:
print("X_train", X_train.shape)

print("y_train", y_train.shape)


X_train (22564, 32, 32, 3)
y_train (22564,)


In [13]:
print(y_train)

[0 0 0 ... 1 0 1]


The idea here is to have a small labelled dataset and a large unlablled set to help improve increasing the accuracy of the model in scenario were we dont have access to large labelled set. We also set aside some data for evaluation test. We are making sure they are equal number of samples for each number exist in each dataset

In [14]:
data_0 = [ x for x,y in zip(X_train,y_train) if y == 0 ] 
data_1 = [ x for x,y in zip(X_train,y_train) if y == 1 ] 

In [15]:
len(data_0)

12565

In [16]:
len(data_1)

9999

In [115]:
source = 0
target = 2000
x_train_shorten = data_0[source:target] + data_1[source:target] 
y_train_shorten = list(np.zeros(target)) + list(np.ones(target)) 

x_train_shorten = np.array( x_train_shorten )
y_train_shorten = np.array( y_train_shorten )

In [116]:
source = 2000
target = 7000
x_train_unlabelled = data_0[source:target] + data_1[source:target] 
y_train_unlabelled = list(np.zeros(target - source)) + list(np.ones(target - source)) 

x_train_unlabelled = np.array( x_train_unlabelled )
y_train_unlabelled = np.array( y_train_unlabelled )

In [117]:
source = 7000
target = 9500
x_eval_shorten = data_0[source:target] + data_1[source:target] 
y_eval_shorten =  list(np.zeros(target - source)) +list(np.ones(target - source)) 

x_eval_shorten = np.array( x_eval_shorten )
y_eval_shorten = np.array( y_eval_shorten )

In [118]:
batch_size = 256

train_dataset = tf.data.Dataset.from_tensor_slices((x_train_shorten,y_train_shorten))
train_dataset = train_dataset.shuffle(1000).batch(batch_size)

pseudo_dataset = tf.data.Dataset.from_tensor_slices((x_train_unlabelled))
pseudo_dataset = pseudo_dataset.shuffle(1000).batch(batch_size)

eval_dataset = tf.data.Dataset.from_tensor_slices((x_eval_shorten,y_eval_shorten))
eval_dataset = eval_dataset.shuffle(1000).batch(batch_size)

In [119]:
lr=0.001
opt = Adam(learning_rate=lr)

In [120]:
# Get pretrained model

pretrained_model= tf.keras.applications.VGG19(include_top=False,
                   input_shape=(32,32,3),
                   #pooling='avg',
                   weights='imagenet')

vgg_model_unfreeze_2block = Sequential()
pretrained_model.trainable = True
for layer in pretrained_model.layers[:20]:
    layer.trainable = False
    
        
#for layer in pretrained_model.layers[:-1]: # go through until last layer
    #resnet_model.add(layer)
    

#resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
vgg_model_unfreeze_2block.add(pretrained_model)
#resnet_model.add(tf.keras.layers.BatchNormalization())
vgg_model_unfreeze_2block.add(Flatten())

prediction_layer = tf.keras.layers.Dense(2, activation='softmax')
vgg_model_unfreeze_2block.add(prediction_layer)

#model.compile(optimizer='adam', loss='categorical_crossentropy')
for i, layer in enumerate(pretrained_model.layers):
        print(i, layer.name, "-", layer.trainable)

0 input_10 - False
1 block1_conv1 - False
2 block1_conv2 - False
3 block1_pool - False
4 block2_conv1 - False
5 block2_conv2 - False
6 block2_pool - False
7 block3_conv1 - False
8 block3_conv2 - False
9 block3_conv3 - False
10 block3_conv4 - False
11 block3_pool - False
12 block4_conv1 - False
13 block4_conv2 - False
14 block4_conv3 - False
15 block4_conv4 - False
16 block4_pool - False
17 block5_conv1 - False
18 block5_conv2 - False
19 block5_conv3 - False
20 block5_conv4 - True
21 block5_pool - True


In [80]:
test_image_data = []
test_image_labels = []
path = data_dir + '/TEST/' + 'O'
images = os.listdir(path)
for img in images:
    image = cv2.imread(path + '/' + img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((image_size, image_size))
    test_image_data.append(np.array(resize_image))
    test_image_labels.append(0)
    
path = data_dir + '/TEST/' + 'R'
images = os.listdir(path)
for img in images:
    image = cv2.imread(path + '/' + img)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((image_size, image_size))
    test_image_data.append(np.array(resize_image))
    test_image_labels.append(1)
# Changing the list to numpy array
test_image_data = np.array(test_image_data)
test_image_labels = np.array(test_image_labels)
print(test_image_data.shape, test_image_labels.shape)

(2513, 32, 32, 3) (2513,)


In [122]:
vgg_model_unfreeze_2block.compile(optimizer=opt,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])

In [123]:
vgg_model_unfreeze_2block.fit(train_dataset, validation_data=eval_dataset, batch_size=256, epochs=100)

Epoch 1/100
16/16 [==============================] - 2s 78ms/step - loss: 4.4279 - accuracy: 0.6518 - val_loss: 2.1225 - val_accuracy: 0.5000
Epoch 2/100
16/16 [==============================] - 1s 66ms/step - loss: 1.5883 - accuracy: 0.4988 - val_loss: 0.6256 - val_accuracy: 0.6108
Epoch 3/100
16/16 [==============================] - 1s 65ms/step - loss: 0.6634 - accuracy: 0.5970 - val_loss: 0.5386 - val_accuracy: 0.7570
Epoch 4/100
16/16 [==============================] - 1s 64ms/step - loss: 0.5659 - accuracy: 0.7113 - val_loss: 0.4844 - val_accuracy: 0.7748
Epoch 5/100
16/16 [==============================] - 1s 65ms/step - loss: 0.5241 - accuracy: 0.7405 - val_loss: 0.4593 - val_accuracy: 0.7882
Epoch 6/100
16/16 [==============================] - 1s 67ms/step - loss: 0.4984 - accuracy: 0.7580 - val_loss: 0.4440 - val_accuracy: 0.7988
Epoch 7/100
16/16 [==============================] - 1s 66ms/step - loss: 0.4884 - accuracy: 0.7635 - val_loss: 0.4317 - val_accuracy: 0.8086
Epoch 

In [124]:


# predict
pred_x = vgg_model_unfreeze_2block.predict(test_image_data)
pred=np.argmax(pred_x,axis=1)
    #Accuracy with the test data
final_acc = accuracy_score(test_image_labels, pred)*100
print(final_acc)

82.29208117787505


In [131]:
model=vgg_model_unfreeze_2block

In [132]:
avg_main_loss = tf.keras.metrics.Mean(name='avg_main_loss', dtype=tf.float32)
pseudo_steps = int(x_train_unlabelled.shape[0] / batch_size )
eval_steps = int(x_eval_shorten.shape[0] / batch_size )

epoch = 5

vgg_model_unfreeze_2block_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

x_values = np.array([])
threshold = 0.80

for epoch_idx in range( epoch ):

  print( "Epoch {}/{}".format( epoch_idx + 1, epoch ) )
  model.evaluate(eval_dataset)

  for _ in range(20):

    pseudo_labels = model.predict( x_train_unlabelled ) 
    

    # only accept psuedo labels with certain threshold accuracy in their probability   
    pseudo_labels = np.array( [ np.argmax(x) if np.max(x) > threshold else -1 for x in pseudo_labels])
    dataset = np.array( [ [x,y] for x,y in zip( x_train_unlabelled, pseudo_labels ) if y != -1 ])
    x_values = np.array( [x for x,y in dataset ])
    y_values = np.array( [y for x,y in dataset ])
    
   

    pseudo_dataset = tf.data.Dataset.from_tensor_slices((x_values, y_values ))
    pseudo_dataset = pseudo_dataset.shuffle(1000).batch(batch_size)


    model.fit(pseudo_dataset, batch_size=256, epochs=1, verbose=0)
    model.fit(train_dataset, batch_size=256, epochs=1, verbose=0)

Epoch 1/5
20/20 [==============================] - 1s 23ms/step - loss: 0.7115 - accuracy: 0.8376


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


Epoch 2/5
20/20 [==============================] - 1s 23ms/step - loss: 0.7976 - accuracy: 0.8386
Epoch 3/5
20/20 [==============================] - 1s 23ms/step - loss: 0.9947 - accuracy: 0.8368
Epoch 4/5
20/20 [==============================] - 1s 23ms/step - loss: 1.1888 - accuracy: 0.8370
Epoch 5/5
20/20 [==============================] - 1s 22ms/step - loss: 1.3601 - accuracy: 0.8354


In [133]:
# predict
pred_x = vgg_model_unfreeze_2block.predict(test_image_data)
pred=np.argmax(pred_x,axis=1)
    #Accuracy with the test data
final_acc = accuracy_score(test_image_labels, pred)*100
print(final_acc)

79.824910465579
